In [1]:
import sys
sys.path.append("../src/initialization/")
import setup_db

setup_db.setup_db_titanic()

Copying Titanic Dataset into SQL-DB
done.


1

In [2]:
# Connect to Postgres
import os
import sqlalchemy
import pandas as pd

engine = sqlalchemy.create_engine(
            "postgresql+psycopg2://"
            + os.getenv("SQL_USER")
            + ":"
            + os.getenv("SQL_PASSWORD")
            + "@"
            + os.getenv("SQL_HOST")
            + ":"
            + os.getenv("SQL_PORT")
            + "/"
            + os.getenv("SQL_DB")
        )
titanic_data = pd.read_sql_table("titanic", engine)
titanic_data

,index,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,None,C,None,328.0,None
1305,1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,None,C,None,NaN,None
1306,1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,None,C,None,304.0,None
1307,1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,None,C,None,NaN,None


In [3]:
# Connect to PySpark
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("TitanicExample") \
    .config("spark.jars", "/srv/KaggleChallenge/plugins/postgresql-42.5.1.jar") \
    .getOrCreate()

df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://" + os.getenv("SQL_HOST") +":" + os.getenv("SQL_PORT") + "/" + os.getenv("SQL_DB")) \
    .option("dbtable", "titanic") \
    .option("user", os.getenv("SQL_USER")) \
    .option("password", os.getenv("SQL_PASSWORD")) \
    .option("driver", "org.postgresql.Driver") \
    .load()

df.printSchema()

/.venv/lib/python3.10/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


22/12/30 09:19:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


root
 |-- index: long (nullable = true)
 |-- pclass: long (nullable = true)
 |-- survived: long (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- sibsp: long (nullable = true)
 |-- parch: long (nullable = true)
 |-- ticket: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- cabin: string (nullable = true)
 |-- embarked: string (nullable = true)
 |-- boat: string (nullable = true)
 |-- body: double (nullable = true)
 |-- home.dest: string (nullable = true)



In [4]:
# Connect to mlflow
import mlflow

mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.set_experiment("my-experiment")


<Experiment: artifact_location='mlflow-artifacts:/852647952488994303', creation_time=1672387233398, experiment_id='852647952488994303', last_update_time=1672387233398, lifecycle_stage='active', name='my-experiment', tags={}>

In [5]:
# Connect to Kafka
import kafka

In [6]:
# Connect to MinIO
import minio
from minio import Minio

# Create a client with the MinIO server playground, its access key
# and secret key.
client = Minio(
    endpoint="minio:9000",
    access_key="GArH6uh55RJ44MBc",
    secret_key="wZki2TznKq0w39wprAqm7KvKCBPeH5nm",
    secure=False
)

found = client.bucket_exists("titanic")
if not found:
    client.make_bucket("titanic")
else:
    print("Bucket 'titanic' already exists")

client.fput_object(
    "titanic", "00_Intro.ipynb", "/srv/KaggleChallenge/extra_data/00_Intro.ipynb",
)
print(
    "'00_intro.ipynb' is successfully uploaded as "
    "object 00_intro.ipynb' to bucket '00_intro.ipynb'."
)

Bucket 'titanic' already exists
'00_intro.ipynb' is successfully uploaded as object 00_intro.ipynb' to bucket '00_intro.ipynb'.


KeyboardInterrupt: 

In [7]:
#import logging
#logging.basicConfig(level=logging.DEBUG)

In [7]:

from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='kafka:9092')
for _ in range(10):
    producer.send('foobar', b'some_message_bytes')

In [ ]:
from kafka import KafkaConsumer

consumer = KafkaConsumer('my_favorite_topic', bootstrap_servers='kafka:9092')
for msg in consumer:
    print (msg)